### Importations

In [1]:
import pandas as pd
import numpy as np
import re
from collections import OrderedDict
from nltk.tokenize import word_tokenize
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer

### Lecture du dataset

In [2]:
df = pd.read_csv("Ressources/RessourcesAnalyse/DatasetClean.csv")
df.head()

,Maladies,Symptomes,Traitement,Examens
0,endométriose,"douleurs pelviennes intenses,douleurs pendant ...","médicaments anti-inflammatoires,chirurgie,trai...","imagerie médicale (échographie pelvienne),lapa..."
1,anémie ferriprive,"fatigue intense,faiblesse générale,palpitation...","suppléments de fer,ajustement du régime alimen...","analyse de sang (taux de fer),numération sangu..."
2,trouble bipolaire,"variations extrêmes d'humeur,irritabilité ,ins...","médicaments stabilisateurs de l'humeur,thérapi...","évaluation psychiatrique,entretiens cliniques"
3,maladie de lyme,éruption cutanée en forme de cible (érythème m...,"antibiotiques,médicaments anti-inflammatoires,...","test sanguin (détection d'anticorps),analyse d..."
4,schizophrénie,"hallucinations auditives,hallucinations olfact...","antipsychotiques,thérapie psychosociale,hospit...","évaluation psychiatrique,entretiens cliniques,..."


## I - Nettoyage de données 

In [3]:
stop_words = stopwords.words('french')
lemmatizer = WordNetLemmatizer()

ps = PorterStemmer()

In [4]:
new_stopwords =["facilement","tres","pendant"]

stop_words.extend(new_stopwords)

In [5]:
def Cleaning(text):
    
    def remove_s(word):
        if word.endswith('s'):
            return word[:-1]
        if word.endswith('ment'):
            return word[:-4]
        else:
            return word
        
    liste2 = ["abdomin ","abdomen","abdominal"]
    liste1 = ["douleur","mal"]
        
    # Supprimer des ponctuations
    text = re.sub("[^\w\s]", " ", text)
   
    text = ' '.join([lemmatizer.lemmatize(unidecode(word.lower())) for word in word_tokenize(text) if word.isalnum() and word.lower() not in stop_words])
    
    # supprimer les chiffres
    text = re.sub("\d", "", text)
    
    #text = [remove_s(word) for word in text]
    
    text = text.replace("ee ","e ")
    text = text.replace("cutane","peau")
    
    for i in liste2:
        text = text.replace(i,"ventre ")
        
    text = list(OrderedDict.fromkeys(text.split()))
    
    text = ' '.join([remove_s(word) for word in text])
    
    for i in liste1:
        text = text.replace(i,"maux")
    
    text = text.replace(" e "," ")
    return text



In [6]:
df["Symptomes_lem_token"] = df["Symptomes"].apply(lambda x: Cleaning(x))
df["Symptomes_lem_token"]

0      maux pelvienne intense regle rapport sexuel lo...
1      fatigue intense faiblesse generale palpitation...
2      variation extreme humeur irritabilite insomnie...
3      eruption peau forme cible erytheme migrant fat...
4      hallucination auditive olfactive visuelle idee...
                             ...                        
96                      fievre frisson maux tete fatigue
97             fievre toux eruption peau maux musculaire
98               fievre fatigue maux tete ganglion enfle
99      fievre fatigue gonfle paupiere trouble cardiaque
100    fievre prolonge perte poid anemie enflure rate...
Name: Symptomes_lem_token, Length: 101, dtype: object

## III -  Calcul de similarité

In [7]:
liste_symptomes ="j'ai la fievres douala poid peau perte fatigue//. frissons yaya essouflement yaounde maux facilement , la ,   tête"
liste_symptomes = Cleaning(liste_symptomes)

In [8]:

import spacy

# Charger le modèle linguistique français avec des vecteurs de mots
nlp = spacy.load('fr_core_news_md')

In [9]:
similarite = []

for i in range(len(df["Symptomes_lem_token"])):
    
    liste_symptome = " ".join([i for i in df['Symptomes_lem_token'][i].split()  if i in liste_symptomes.split()])
    
    val = nlp(liste_symptome)
    
    doc = nlp(df['Symptomes_lem_token'][i])
    
    similarity = doc.similarity(val)
    
    similarite.append(similarity)

/var/folders/hj/yplghdzn6nbbj6yf39wxn2240000gn/T/ipykernel_10826/3289238199.py:11: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = doc.similarity(val)


In [10]:
similarite = np.array(similarite)

In [11]:
df['Similarite_%'] = 100*similarite.flatten()

In [12]:
df['Similarite_%']

0       63.977452
1       77.056196
2       64.116313
3       80.123305
4        0.000000
          ...    
96     100.000000
97      92.449499
98      94.371407
99      80.610482
100     87.203798
Name: Similarite_%, Length: 101, dtype: float64

In [13]:
df_tri = df.sort_values(by='Similarite_%', ascending=False)

In [14]:
print(liste_symptomes)
df_tri

fievre douala poid peau perte fatigue frisson yaya essoufle yaounde maux tete


,Maladies,Symptomes,Traitement,Examens,Symptomes_lem_token,Similarite_%
96,paludisme,"fièvre,frissons,maux de tête,fatigue","antipaludiques,repos",test de dépistage du paludisme,fievre frisson maux tete fatigue,100.000000
84,varicelle,"éruption cutanée avec des démangeaisons,fièvre...","antihistaminiques,antipyrétiques,calamine pour...",les symptômes sont souvent suffisants pour le ...,eruption peau demangeaison fievre maux tete fa...,95.949553
91,cancer,"perte de poids inexpliquée,bosse anormale,fati...","chirurgie,chimiothérapie,radiothérapie","biopsie,imagerie médicale",perte poid inexplique bosse anormauxe fatigue ...,95.833342
44,maladie cœliaque,"diarrhée chronique,douleurs abdominales,ballon...","régime sans gluten,suppléments nutritionnels,m...","analyse de sang (anticorps),biopsie de l'intes...",diarrhe chronique maux ventre ballonnement fat...,95.035814
60,mésothéliome,"essoufflement,douleur thoracique,toux persista...","chirurgie,chimiothérapie,thérapie ciblée","scanner thoracique,biopsie,imagerie médicale",essouffle maux thoracique toux persistante per...,94.386656
...,...,...,...,...,...,...
62,ostéosarcome,"douleur osseuse, généralement dans les bras ou...","chirurgie,chimiothérapie,radiothérapie","radiographie,irm,biopsie",maux osseuse generale bra jambe gonfle autour ...,41.325473
87,coqueluche,"toux sévère par accès,éternuements,épuisement,...","antibiotiques,vaccin de rappel",prélèvement de sécrétions nasales pour la culture,toux severe acce eternuement epuise vomissemen...,0.000000
4,schizophrénie,"hallucinations auditives,hallucinations olfact...","antipsychotiques,thérapie psychosociale,hospit...","évaluation psychiatrique,entretiens cliniques,...",hallucination auditive olfactive visuelle idee...,0.000000
22,condylomes génitaux (hpv),"verrues génitales,démangeaisons,saignements pe...","crèmes antivirales,ablation des verrues","test hpv,examen visuel",verrue genitale demangeaison saignement rappor...,0.000000
